In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import *

In [0]:
# Load AWS Credentials
spark.conf.set("fs.s3a.access.key", "")
spark.conf.set("fs.s3a.secret.key", "")

In [0]:
# Read HVFHV DataFrames and drop duplicate rows
HVFHV_df = spark.read.option("recursiveFileLookup", "true").parquet("s3a://capstone-techcatalyst-conformed/group4/hvfhv_data/*",multiLine=True, encoding="utf8")

In [0]:
# Calculate Trip Distance % of Total Trip Distance for the day.
HVFHV_df = HVFHV_df.withColumn('trip_distance_percentage', col('trip_miles')/ sum('trip_miles').over(Window.partitionBy(to_date("Pickup_datetime")))*100)

# Calculate Trip Duration % of Total Time Spent Active for the day.
HVFHV_df = HVFHV_df.withColumn('trip_duration_percentage', col('trip_time')/ sum('trip_time').over(Window.partitionBy(to_date("Pickup_datetime")))*100)

# Calculate the amount of time it takes to be picked up after requesting a ride
HVFHV_df = HVFHV_df.withColumn('pickup_wait_time', (unix_timestamp('pickup_datetime') - unix_timestamp('request_datetime'))/60)

In [0]:
HVFHV_parquet_path = f"s3a://capstone-techcatalyst-transformed/group4/hvfhv_data/"

HVFHV_df.write.partitionBy("year", "month", "day_of_month").parquet(HVFHV_parquet_path)